In [ ]:
import ipyleaflet as ileaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd
import pandas as pd

# Load the GeoJSON data
geojson_path = '/home/hantaox/2024-winter-rafi-poultry-cafos/output/final_data_ne.geojson'
poultry_barns = gpd.read_file(geojson_path)

# If 'check' column doesn't exist, initialize it
if 'check' not in poultry_barns.columns:
    poultry_barns['check'] = 0  # Unchecked

# Filter out already checked locations
poultry_barns = poultry_barns[poultry_barns['check'] == 0]
poultry_barns = poultry_barns[poultry_barns['false_positive'] != 1]

# Initialize DataFrame for recording responses
verification_results = pd.DataFrame(columns=['index', 'longitude', 'latitude', 'classification'])

# Create and display the map initially
m = ileaflet.Map(zoom=18, basemap=ileaflet.basemaps.Esri.WorldImagery)

# Function to handle classification and move to the next location
def handle_classification(button):
    index = button.index
    centroid = button.centroid
    classification = button.description

    # Mark the barn as checked and update the DataFrame with results
    poultry_barns.at[index, 'check'] = 1
    verification_results.loc[len(verification_results)] = [index, centroid[1], centroid[0], classification]
    clear_output(wait=False)
    print(f"Barns remaining: {len(poultry_barns[poultry_barns['check'] == 0])}")

    # Check and display next location
    next_index = index + 1
    if next_index < len(poultry_barns):
        display_location(next_index)
    else:
        print("All locations have been reviewed.")

# Function to save the updated GeoJSON file
def end_check(button):
    poultry_barns.to_file(geojson_path, driver='GeoJSON')
    print("Verification data saved and updated!")

# Display function for each location
def display_location(index):
    feature = poultry_barns.iloc[index]
    coords = feature.geometry.exterior.coords.xy
    centroid = (sum(coords[1]) / len(coords[1]), sum(coords[0]) / len(coords[0]))

    # Update map
    m.center = centroid
    m.layers = [m.layers[0]]
    polygon = ileaflet.Polygon(locations=[(y, x) for x, y in zip(*coords)], color="red", fill_color="red")
    m.add_layer(polygon)

    # Buttons with updated layout properties for size and alignment
    button_style = widgets.Layout(width='200px', height='50px', margin='5px')
    btn_true_positive = widgets.Button(description='True Positive', button_style='success', layout=button_style)
    btn_true_negative = widgets.Button(description='True Negative', button_style='warning', layout=button_style)
    btn_end_check = widgets.Button(description='End the Check', button_style='info', layout=button_style)
    btn_true_positive.on_click(handle_classification)
    btn_true_negative.on_click(handle_classification)
    btn_end_check.on_click(end_check)
    btn_true_positive.index = index
    btn_true_negative.index = index
    btn_true_positive.centroid = centroid
    btn_true_negative.centroid = centroid
    btn_end_check.index = index

    # Vertical box for buttons aligned to the center-right of the map
    button_box = widgets.VBox([btn_true_positive, btn_true_negative, btn_end_check], layout=widgets.Layout(align_items='center'))
    display(widgets.HBox([m, button_box], layout=widgets.Layout(align_items='center', justify_content='space-between')))

# Start the process with the first location
display_location(0)


In [ ]:
import ipyleaflet as ileaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd
import pandas as pd

# Load the GeoJSON data
geojson_path = '/home/hantaox/2024-winter-rafi-poultry-cafos/output/final_data_ne.geojson'
poultry_barns = gpd.read_file(geojson_path)

# Filter out already false positives
poultry_barns = poultry_barns[(poultry_barns['false_positive'] != 1)]


# List to store indices of checked barns
checked_indices = []

# Create and display the map initially
m = ileaflet.Map(zoom=18, basemap=ileaflet.basemaps.Esri.WorldImagery)

# Function to handle classification and move to the next location
def handle_classification(button):
    index = button.index
    centroid = button.centroid
    classification = button.description

    # Update the list with checked index
    checked_indices.append(index)
    clear_output(wait=True)
    print(f"Barns remaining: {len(poultry_barns) - len(checked_indices)}")

    # Move to the next location
    next_index = index + 1
    if next_index < len(poultry_barns):
        display_location(next_index)
    else:
        print("All locations have been reviewed.")

# Display function for each location
def display_location(index):
    feature = poultry_barns.iloc[index]
    coords = feature.geometry.exterior.coords.xy
    centroid = (sum(coords[1]) / len(coords[1]), sum(coords[0]) / len(coords[0]))

    # Update map
    m.center = centroid
    m.layers = [m.layers[0]]
    polygon = ileaflet.Polygon(locations=[(y, x) for x, y in zip(*coords)], color="red", fill_color="red")
    m.add_layer(polygon)

    # Buttons
    btn_true_positive = widgets.Button(description='True Positive', button_style='success', layout=widgets.Layout(width='200px', height='50px', margin='5px'))
    btn_true_negative = widgets.Button(description='True Negative', button_style='warning', layout=widgets.Layout(width='200px', height='50px', margin='5px'))
    btn_true_positive.on_click(handle_classification)
    btn_true_negative.on_click(handle_classification)
    btn_true_positive.index = index
    btn_true_negative.index = index
    btn_true_positive.centroid = centroid
    btn_true_negative.centroid = centroid

    # Layout
    button_box = widgets.VBox([btn_true_positive, btn_true_negative], layout=widgets.Layout(align_items='center'))
    display(widgets.HBox([m, button_box], layout=widgets.Layout(align_items='center', justify_content='space-between')))

# Start the process with the first location if any are left
if not poultry_barns.empty:
    display_location(poultry_barns.index[0])
else:
    print("All locations have been reviewed.")


In [ ]:
print(poultry_barns.index)

In [ ]:
checked_indices

In [ ]:
with open('/home/hantaox/2024-winter-rafi-poultry-cafos/output/checked_barns_indices.txt', 'w') as file:
    file.write('\n'.join(map(str, checked_indices)))
print("Checked barn indices saved!")

In [1]:
import ipyleaflet as ileaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd
import pandas as pd
import os

# Load the GeoJSON data
geojson_path = '../output/final_data_ne.geojson'
poultry_barns = gpd.read_file(geojson_path)

if os.path.exists('../output/checked_barns_indices.txt'):
    with open('../output/checked_barns_indices.txt', 'r') as file:
        lines = file.readlines()
    checked_indices = [(int(line.strip().split(',')[0]), int(line.strip().split(',')[1])) for line in lines]
    print(checked_indices)
else:
    # Initialize a list to keep track of checked indices
    checked_indices = []

# Filter out already false positives
poultry_barns = poultry_barns[(poultry_barns['false_positive'] != 1)]

already_check_lst = [i[0] for i in checked_indices]

poultry_barns = poultry_barns[~poultry_barns.index.isin(already_check_lst)]

# Create and display the map initially
m = ileaflet.Map(zoom=18, basemap=ileaflet.basemaps.Esri.WorldImagery)

def handle_classification(button, result):
    index = button.index
    centroid = button.centroid
    classification = button.description

    # Update the list with checked index and save to file
    checked_indices.append((index, result))
    with open('../output/checked_barns_indices.txt', 'a') as file:
        file.write(f'{index}, {result}\n')
    
    clear_output(wait=True)
    print(f"Barns remaining: {len(poultry_barns) - len(checked_indices)}")

    # Move to the next location
    next_index = index + 1
    if next_index < len(poultry_barns):
        display_location(next_index)
    else:
        print("All locations have been reviewed.")

# Display function for each location
def display_location(index):
    feature = poultry_barns.iloc[index]
    coords = feature.geometry.exterior.coords.xy
    centroid = (sum(coords[1]) / len(coords[1]), sum(coords[0]) / len(coords[0]))

    # Update map
    m.center = centroid
    m.layers = [m.layers[0]]
    polygon = ileaflet.Polygon(locations=[(y, x) for x, y in zip(*coords)], color="red", fill_color="red")
    m.add_layer(polygon)

    # Buttons with updated layout properties for size and alignment
    button_style = widgets.Layout(width='200px', height='50px', margin='5px')
    btn_true_positive = widgets.Button(description='True Positive', button_style='success', layout=button_style)
    btn_true_negative = widgets.Button(description='True Negative', button_style='warning', layout=button_style)
    btn_true_positive.on_click(lambda b: handle_classification(b, 1))
    btn_true_negative.on_click(lambda b: handle_classification(b, 0))
    btn_true_positive.index = index
    btn_true_negative.index = index
    btn_true_positive.centroid = centroid
    btn_true_negative.centroid = centroid

    # Vertical box for buttons aligned to the center-right of the map
    button_box = widgets.VBox([btn_true_positive, btn_true_negative], layout=widgets.Layout(align_items='center'))
    display(widgets.HBox([m, button_box], layout=widgets.Layout(align_items='center', justify_content='space-between')))

# Start the process with the first location if any are left
if not poultry_barns.empty:
    display_location(poultry_barns.index[0])
else:
    print("All locations have been reviewed.")


Barns remaining: 4200
